# ChatKit + OpenAI Responses API Integration

This notebook demonstrates how to integrate OpenAI's ChatKit UI library with the Responses API to build a modern chat interface.

## What is ChatKit?

ChatKit is OpenAI's framework-agnostic, drop-in chat solution for building high-quality, AI-powered chat experiences. It provides:

- 🎨 **Polished UI Components** - Ready-to-use chat interface
- 🔄 **Streaming Support** - Real-time response streaming
- 🛠️ **Tool Integration** - Support for function calling and widgets
- 📎 **Attachments** - File and image upload handling
- 🎯 **Framework Agnostic** - Works with React, Vue, Angular, or vanilla JS

## Architecture Overview

```
┌─────────────────┐
│  React Frontend │
│   (ChatKit UI)  │
└────────┬────────┘
         │
         │ /api/chatkit/session
         │ /api/chat
         ▼
┌─────────────────┐
│ FastAPI Backend │
│ (Python Server) │
└────────┬────────┘
         │
         │ Responses API
         ▼
┌─────────────────┐
│  OpenAI API     │
└─────────────────┘
```

## Setup Requirements

### Backend (Python)
```bash
pip install fastapi uvicorn openai python-dotenv
```

### Frontend (Node.js)
```bash
npm install @openai/chatkit-react react react-dom
```

## Part 1: Backend Implementation with Responses API

Let's create a FastAPI backend that uses the OpenAI Responses API to handle chat messages.

### Backend Code: `backend/app.py`

```python
from fastapi import FastAPI, HTTPException
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
from openai import OpenAI
import os
from typing import List, Dict, Optional
import uuid
from dotenv import load_dotenv

load_dotenv()

app = FastAPI()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Enable CORS for frontend
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# In-memory storage for conversation history
conversations: Dict[str, List[Dict]] = {}


class ChatKitSessionRequest(BaseModel):
    """Request model for creating a ChatKit session"""
    existing_session_id: Optional[str] = None


class ChatKitSessionResponse(BaseModel):
    """Response model for ChatKit session"""
    client_secret: str
    session_id: str


class ChatMessage(BaseModel):
    """Chat message model"""
    role: str
    content: str


class ChatRequest(BaseModel):
    """Request model for chat endpoint"""
    session_id: str
    message: str
    messages: Optional[List[ChatMessage]] = None


class ChatResponse(BaseModel):
    """Response model for chat endpoint"""
    response: str
    session_id: str


@app.post("/api/chatkit/session")
async def create_chatkit_session(
    request: Optional[ChatKitSessionRequest] = None
) -> ChatKitSessionResponse:
    """
    Create or retrieve a ChatKit session.
    
    ChatKit requires a client_secret for authentication. In a production app,
    this would be a secure token. For this demo, we use the session_id.
    """
    session_id = request.existing_session_id if request else None
    
    if not session_id:
        # Create new session
        session_id = str(uuid.uuid4())
        conversations[session_id] = []
    
    # In production, generate a secure token here
    # For demo purposes, we'll use the session_id as the client_secret
    client_secret = f"demo_secret_{session_id}"
    
    return ChatKitSessionResponse(
        client_secret=client_secret,
        session_id=session_id
    )


@app.post("/api/chat")
async def chat(request: ChatRequest) -> ChatResponse:
    """
    Handle chat messages using OpenAI's Responses API.
    
    This endpoint:
    1. Retrieves or creates conversation history
    2. Adds the user's message to history
    3. Calls OpenAI's Responses API
    4. Stores the assistant's response
    5. Returns the response to the frontend
    """
    session_id = request.session_id
    
    # Get or create conversation history
    if session_id not in conversations:
        conversations[session_id] = []
    
    # Add user message to history
    conversations[session_id].append({
        "role": "user",
        "content": request.message
    })
    
    try:
        # Call OpenAI Responses API
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=conversations[session_id],
            max_tokens=1000,
            temperature=0.7
        )
        
        # Extract assistant's response
        assistant_message = response.choices[0].message.content
        
        # Add assistant response to history
        conversations[session_id].append({
            "role": "assistant",
            "content": assistant_message
        })
        
        return ChatResponse(
            response=assistant_message,
            session_id=session_id
        )
    
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))


@app.get("/api/conversations/{session_id}")
async def get_conversation(session_id: str):
    """
    Retrieve conversation history for a session.
    Useful for debugging or resuming conversations.
    """
    if session_id not in conversations:
        raise HTTPException(status_code=404, detail="Session not found")
    
    return {"session_id": session_id, "messages": conversations[session_id]}


@app.delete("/api/conversations/{session_id}")
async def delete_conversation(session_id: str):
    """
    Clear conversation history for a session.
    """
    if session_id in conversations:
        del conversations[session_id]
        return {"message": "Conversation deleted"}
    
    raise HTTPException(status_code=404, detail="Session not found")


if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=8000)
```

## Part 2: Frontend Implementation with ChatKit

Now let's create a React frontend using ChatKit components.

### Frontend Code: `frontend/src/App.jsx`

```jsx
import React, { useState } from 'react';
import { ChatKit, useChatKit } from '@openai/chatkit-react';
import '@openai/chatkit-react/styles.css';

function App() {
  const [sessionId, setSessionId] = useState(null);
  
  const { control } = useChatKit({
    api: {
      // Get client secret from backend
      async getClientSecret(existingSessionId) {
        const response = await fetch('http://localhost:8000/api/chatkit/session', {
          method: 'POST',
          headers: {
            'Content-Type': 'application/json',
          },
          body: JSON.stringify({
            existing_session_id: existingSessionId,
          }),
        });
        
        const data = await response.json();
        setSessionId(data.session_id);
        return data.client_secret;
      },
      
      // Send messages to backend
      async sendMessage(message) {
        if (!sessionId) {
          throw new Error('No session ID available');
        }
        
        const response = await fetch('http://localhost:8000/api/chat', {
          method: 'POST',
          headers: {
            'Content-Type': 'application/json',
          },
          body: JSON.stringify({
            session_id: sessionId,
            message: message.content,
          }),
        });
        
        const data = await response.json();
        
        // Return in ChatKit expected format
        return {
          role: 'assistant',
          content: data.response,
        };
      },
    },
    
    // Optional configuration
    config: {
      model: 'gpt-4o-mini',
      welcomeMessage: 'Hello! How can I help you today?',
    },
  });

  return (
    <div className="App" style={{
      display: 'flex',
      justifyContent: 'center',
      alignItems: 'center',
      height: '100vh',
      backgroundColor: '#f5f5f5',
      padding: '20px',
    }}>
      <div style={{
        width: '100%',
        maxWidth: '800px',
        height: '600px',
        boxShadow: '0 4px 6px rgba(0, 0, 0, 0.1)',
        borderRadius: '12px',
        overflow: 'hidden',
      }}>
        <ChatKit 
          control={control} 
          style={{ height: '100%', width: '100%' }}
        />
      </div>
    </div>
  );
}

export default App;
```

### Alternative: Simple HTML/Vanilla JS Implementation

If you prefer not to use React, here's a vanilla JavaScript version:

```html
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <title>ChatKit + Responses API Demo</title>
  <script src="https://cdn.platform.openai.com/deployments/chatkit/chatkit.js" async></script>
  <style>
    body {
      margin: 0;
      padding: 20px;
      font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', sans-serif;
      background: #f5f5f5;
      display: flex;
      justify-content: center;
      align-items: center;
      min-height: 100vh;
    }
    #chat-container {
      width: 100%;
      max-width: 800px;
      height: 600px;
      background: white;
      border-radius: 12px;
      box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);
    }
  </style>
</head>
<body>
  <div id="chat-container"></div>

  <script>
    let sessionId = null;
    
    // Initialize ChatKit when the library loads
    window.addEventListener('load', async () => {
      if (!window.ChatKit) {
        console.error('ChatKit library not loaded');
        return;
      }
      
      const chatKit = new window.ChatKit({
        container: document.getElementById('chat-container'),
        
        api: {
          async getClientSecret(existingSessionId) {
            const response = await fetch('http://localhost:8000/api/chatkit/session', {
              method: 'POST',
              headers: { 'Content-Type': 'application/json' },
              body: JSON.stringify({ existing_session_id: existingSessionId }),
            });
            
            const data = await response.json();
            sessionId = data.session_id;
            return data.client_secret;
          },
          
          async sendMessage(message) {
            if (!sessionId) {
              throw new Error('No session ID available');
            }
            
            const response = await fetch('http://localhost:8000/api/chat', {
              method: 'POST',
              headers: { 'Content-Type': 'application/json' },
              body: JSON.stringify({
                session_id: sessionId,
                message: message.content,
              }),
            });
            
            const data = await response.json();
            return {
              role: 'assistant',
              content: data.response,
            };
          },
        },
        
        config: {
          welcomeMessage: 'Hello! How can I help you today?',
        },
      });
    });
  </script>
</body>
</html>
```

## Part 3: Running the Application

### Step 1: Start the Backend

Create a `.env` file in the backend directory:
```
OPENAI_API_KEY=your_openai_api_key_here
```

Run the FastAPI server:
```bash
cd backend
python app.py
```

The backend will start at `http://localhost:8000`

### Step 2: Start the Frontend

#### For React Version:
```bash
cd frontend
npm install
npm run dev
```

#### For HTML Version:
Simply open the HTML file in your browser or use a simple HTTP server:
```bash
python -m http.server 3000
```

## Part 4: Testing the Integration

Let's write a simple test to verify our backend is working:

In [ ]:
import requests
import json

# Base URL for the API
BASE_URL = "http://localhost:8000"

def test_chatkit_integration():
    """Test the ChatKit + Responses API integration"""
    
    print("🧪 Testing ChatKit Integration...\n")
    
    # Step 1: Create a session
    print("1️⃣ Creating session...")
    session_response = requests.post(f"{BASE_URL}/api/chatkit/session")
    session_data = session_response.json()
    session_id = session_data["session_id"]
    print(f"   ✅ Session created: {session_id}\n")
    
    # Step 2: Send a message
    print("2️⃣ Sending message...")
    chat_request = {
        "session_id": session_id,
        "message": "Hello! What is OpenAI's Responses API?"
    }
    chat_response = requests.post(
        f"{BASE_URL}/api/chat",
        json=chat_request
    )
    chat_data = chat_response.json()
    print(f"   📤 User: {chat_request['message']}")
    print(f"   📥 Assistant: {chat_data['response'][:100]}...\n")
    
    # Step 3: Get conversation history
    print("3️⃣ Retrieving conversation history...")
    history_response = requests.get(
        f"{BASE_URL}/api/conversations/{session_id}"
    )
    history_data = history_response.json()
    print(f"   ✅ Messages in conversation: {len(history_data['messages'])}\n")
    
    # Step 4: Send a follow-up message
    print("4️⃣ Sending follow-up message...")
    followup_request = {
        "session_id": session_id,
        "message": "Can you give me a specific example?"
    }
    followup_response = requests.post(
        f"{BASE_URL}/api/chat",
        json=followup_request
    )
    followup_data = followup_response.json()
    print(f"   📤 User: {followup_request['message']}")
    print(f"   📥 Assistant: {followup_data['response'][:100]}...\n")
    
    print("✅ All tests passed!")

# Uncomment to run the test
# test_chatkit_integration()

## Key Features Demonstrated

### 1. **Session Management**
- Each user gets a unique session ID
- Conversations are stored in memory (can be persisted to database)
- Session IDs enable resuming conversations

### 2. **Conversation History**
- Messages are stored and passed to the Responses API
- Enables context-aware responses
- Can retrieve or delete conversation history via API endpoints

### 3. **OpenAI Responses API Integration**
- Uses `client.chat.completions.create()` instead of Assistants API
- Direct API calls with conversation history
- Simpler architecture compared to Assistants API

### 4. **ChatKit UI**
- Professional chat interface out of the box
- Handles message rendering and input
- Responsive design and mobile-friendly

## Advanced Features to Add

### 1. **Streaming Responses**

Modify the backend to support streaming:

```python
from fastapi.responses import StreamingResponse

@app.post("/api/chat/stream")
async def chat_stream(request: ChatRequest):
    async def generate():
        stream = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=conversations[request.session_id],
            stream=True
        )
        
        for chunk in stream:
            if chunk.choices[0].delta.content:
                yield f"data: {json.dumps({'content': chunk.choices[0].delta.content})}\n\n"
    
    return StreamingResponse(generate(), media_type="text/event-stream")
```

### 2. **Function Calling**

Add tools to the Responses API:

```python
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get current weather",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {"type": "string"}
                },
                "required": ["location"]
            }
        }
    }
]

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=conversations[session_id],
    tools=tools,
    tool_choice="auto"
)
```

### 3. **Persistent Storage**

Replace in-memory storage with a database:

```python
from sqlalchemy import create_engine, Column, String, JSON
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()

class Conversation(Base):
    __tablename__ = "conversations"
    session_id = Column(String, primary_key=True)
    messages = Column(JSON)
```

### 4. **Authentication**

Add user authentication:

```python
from fastapi.security import HTTPBearer

security = HTTPBearer()

@app.post("/api/chat")
async def chat(request: ChatRequest, credentials: HTTPAuthorizationCredentials = Depends(security)):
    # Verify token
    user_id = verify_token(credentials.credentials)
    # ... rest of the code
```

## Comparison: Responses API vs Assistants API

| Feature | Responses API (This Example) | Assistants API |
|---------|----------------------------|----------------|
| **Complexity** | Simple, direct API calls | More complex, thread-based |
| **State Management** | You manage conversation history | OpenAI manages threads |
| **Built-in Tools** | Manual implementation | Code interpreter, file search |
| **Cost** | Pay per token | Pay per token + storage |
| **Control** | Full control over flow | Less control, more automation |
| **Best For** | Simple chatbots, quick prototypes | Complex agents, file operations |

### When to Use Responses API:
- ✅ Simple chat applications
- ✅ Full control over conversation flow
- ✅ Custom state management
- ✅ Lower complexity

### When to Use Assistants API:
- ✅ Need built-in tools (code interpreter, file search)
- ✅ Long-running conversations
- ✅ File operations and analysis
- ✅ OpenAI manages state for you

## Next Steps

1. **Deploy to Production**
   - Use a production ASGI server (gunicorn + uvicorn)
   - Add proper authentication and rate limiting
   - Use a real database (PostgreSQL, MongoDB)
   - Deploy frontend to Vercel/Netlify
   - Deploy backend to Railway/Render/AWS

2. **Add More Features**
   - Implement streaming responses
   - Add function calling/tools
   - Support file uploads
   - Add conversation summarization

3. **Improve UX**
   - Add loading indicators
   - Implement error handling
   - Add conversation search
   - Enable conversation export

4. **Monitor and Optimize**
   - Add logging and monitoring
   - Implement caching
   - Track usage and costs
   - Optimize response times

## Resources

- [ChatKit Documentation](https://github.com/openai/chatkit-js)
- [OpenAI Responses API](https://platform.openai.com/docs/guides/text-generation)
- [FastAPI Documentation](https://fastapi.tiangolo.com/)
- [ChatKit Advanced Examples](https://github.com/openai/openai-chatkit-advanced-samples)

## Conclusion

This notebook demonstrated how to integrate ChatKit with OpenAI's Responses API to build a modern chat interface. The key advantages of this approach are:

- 🚀 **Simple Architecture** - Direct API calls, easy to understand
- 🎨 **Beautiful UI** - Professional ChatKit interface
- 🔧 **Full Control** - Manage your own conversation state
- 💰 **Cost Effective** - Only pay for tokens used
- 🔌 **Flexible** - Easy to add custom features

You can extend this foundation to build sophisticated chat applications with custom tools, integrations, and business logic!